# Dowload data and preprocess

In [ ]:
pip install plotly --upgrade
pip install covidcast

     |████████████████████████████████| 25.3 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import pandas as pd

In [ ]:
from datetime import date
import covidcast
# cases_asis = covidcast.signal("jhu-csse", "confirmed_incidence_num",
#                           date(2020, 2, 18), date(2021, 11, 1),
#                           geo_type="state",geo_values=["il","wi","mi","in","mo","ia","ky"])
# cases_asis = covidcast.signal("jhu-csse", "confirmed_incidence_num",
#                           date(2020, 2, 18), date(2021, 11, 1),
#                           geo_type="state",geo_values=["il","id","sc","az","ga","mo","tn","sd","fl","ok","ne"])
#Washington,Nevada,California,New Mexico,Hawaii,Connecticut,New York
cases_asis = covidcast.signal("jhu-csse", "confirmed_incidence_prop",
                          date(2020,1,22), date(2021, 11, 21),
                          geo_type="state")#,geo_values=["mi","ny","ct","nm","ca","nv","wa"])
cases_asis.geo_value = covidcast.abbr_to_name(cases_asis.geo_value,ignore_case = True)

In [ ]:
cases_asis['time_value'] = pd.to_datetime(cases_asis['time_value'])

In [ ]:
cases_asis.to_csv('covidcasesper100k.csv',index=False)

In [1]:
import pandas as pd
covidcasesper100k = pd.read_csv("/covidcasesper100k.csv")

In [2]:
covidcasesper100k['time_value'] = pd.to_datetime(covidcasesper100k['time_value'])

In [3]:
# Transform daily to weekly
df = pd.DataFrame(covidcasesper100k.groupby(by=[pd.Grouper(key='time_value',freq="W-MON"),covidcasesper100k.geo_value])["value"].sum()).reset_index().set_index('time_value')

In [4]:
dfpivot = df.pivot_table(index='time_value', columns='geo_value', values='value').rename_axis(None,axis=1)

# Michigan

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO
enddate=pd.to_datetime("2021-06-22")

In [ ]:
dfpivot.Michigan.head(10)

time_value
2020-01-27     0.000000
2020-02-03     0.000000
2020-02-10     0.000000
2020-02-17     0.000000
2020-02-24     0.000000
2020-03-02     0.000000
2020-03-09     0.000000
2020-03-16     0.761041
2020-03-23    13.798135
2020-03-30    53.560394
Name: Michigan, dtype: float64

In [ ]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-06-28")]
#dfpivot_cor = dfpivot[(dfpivot.index > "2020-5-4")&(dfpivot.index < "2021-6-30")]#


In [ ]:
pd.to_datetime("2021-07-5")+relativedelta(weeks=+8)

Timestamp('2021-08-30 00:00:00')

In [ ]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
              Alabama     Alaska  ...  Wisconsin    Wyoming
time_value                        ...                      
2021-06-28  29.218544  24.763776  ...   9.875434  80.538803

[1 rows x 52 columns]


In [ ]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Michigan)
mi = mi[mi.Michigan>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Michigan
Alaska,0.975772
Arizona,0.976138
Arkansas,0.976114
California,0.771532
Colorado,0.777116
Connecticut,0.884351
Delaware,0.923689
District of Columbia,0.812698
Florida,0.983827
Georgia,0.805652


In [ ]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [ ]:
set(list(maskmandate.State))

{'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming'}

In [ ]:
# import plotly.graph_objects as go

# # Create traces
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot.Michigan,
#                     mode='lines',
#                     name='Michigan'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["New York"],
#                     mode='lines',
#                     name='New York'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Nebraska"],
#                     mode='lines',
#                     name='Nebraska'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Idaho"],
#                     mode='lines',
#                     name='Idaho'))
# fig.add_vrect(x0="2021-6-14", x1="2021-8-30",
#               fillcolor="green", opacity=0.25, line_width=0)
# fig.update_layout(
#     legend=dict(
#         x=0,
#         y=1),
#     autosize=False,
#     width=800,
#     height=500)
# fig.show()

In [ ]:

final_series = dfpivot[[ 'Michigan','Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming']]



final_series.to_csv("Michigan.csv")

# Illinois

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO

In [ ]:
dfpivot.Illinois.head(10)

time_value
2020-01-27     0.007892
2020-02-03     0.007892
2020-02-10     0.000000
2020-02-17     0.000000
2020-02-24     0.000000
2020-03-02     0.015889
2020-03-09     0.023833
2020-03-16     0.603772
2020-03-23     9.549133
2020-03-30    29.958220
Name: Illinois, dtype: float64

In [ ]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-06-21")]
#dfpivot_cor = dfpivot[(dfpivot.index > "2020-5-4")&(dfpivot.index < "2021-6-30")]#


In [ ]:
pd.to_datetime("2021-06-28")+relativedelta(weeks=+8)

Timestamp('2021-08-23 00:00:00')

In [ ]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
              Alabama     Alaska  ...  Wisconsin    Wyoming
time_value                        ...                      
2021-06-21  23.163519  29.605855  ...   9.326799  68.002912

[1 rows x 52 columns]


In [ ]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Illinois)
mi = mi[mi.Illinois>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Illinois
Alaska,0.976586
Arizona,0.974488
Arkansas,0.980631
California,0.776536
Colorado,0.773740
Connecticut,0.873439
Delaware,0.920515
District of Columbia,0.814403
Florida,0.977666
Georgia,0.812149


In [ ]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [ ]:
set(list(maskmandate.State))

{'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming'}

In [ ]:
# import plotly.graph_objects as go

# # Create traces
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot.Michigan,
#                     mode='lines',
#                     name='Michigan'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["New York"],
#                     mode='lines',
#                     name='New York'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Nebraska"],
#                     mode='lines',
#                     name='Nebraska'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Idaho"],
#                     mode='lines',
#                     name='Idaho'))
# fig.add_vrect(x0="2021-6-14", x1="2021-8-30",
#               fillcolor="green", opacity=0.25, line_width=0)
# fig.update_layout(
#     legend=dict(
#         x=0,
#         y=1),
#     autosize=False,
#     width=800,
#     height=500)
# fig.show()

In [ ]:
final_series = dfpivot[['Illinois','Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming']]



final_series.to_csv("Illinois.csv")

# Illinois reissue

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO

In [ ]:
dfpivot.Illinois.head(10)

time_value
2020-01-27     0.007892
2020-02-03     0.007892
2020-02-10     0.000000
2020-02-17     0.000000
2020-02-24     0.000000
2020-03-02     0.015889
2020-03-09     0.023833
2020-03-16     0.603772
2020-03-23     9.549133
2020-03-30    29.958220
Name: Illinois, dtype: float64

In [ ]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-09-06")]
#dfpivot_cor = dfpivot[(dfpivot.index > "2020-5-4")&(dfpivot.index < "2021-6-30")]#


In [ ]:
pd.to_datetime("2021-09-13")+relativedelta(weeks=+8)

Timestamp('2021-11-08 00:00:00')

In [ ]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
               Alabama      Alaska  ...   Wisconsin    Wyoming
time_value                          ...                       
2021-09-06  612.918904  374.638573  ...  245.137077  420.72509

[1 rows x 52 columns]


In [ ]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Illinois)
mi = mi[mi.Illinois>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Illinois
Alaska,0.970342
Arizona,0.964729
Arkansas,0.969399
California,0.722395
Colorado,0.764787
Connecticut,0.870157
Delaware,0.900997
District of Columbia,0.772964
Florida,0.968666
Georgia,0.807570


In [ ]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [ ]:
set(list(maskmandate.State))

{'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'Wisconsin',
 'Wyoming'}

In [ ]:
# import plotly.graph_objects as go

# # Create traces
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot.Michigan,
#                     mode='lines',
#                     name='Michigan'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["New York"],
#                     mode='lines',
#                     name='New York'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Nebraska"],
#                     mode='lines',
#                     name='Nebraska'))
# fig.add_trace(go.Scatter(x=dfpivot.index, y=dfpivot["Idaho"],
#                     mode='lines',
#                     name='Idaho'))
# fig.add_vrect(x0="2021-6-14", x1="2021-8-30",
#               fillcolor="green", opacity=0.25, line_width=0)
# fig.update_layout(
#     legend=dict(
#         x=0,
#         y=1),
#     autosize=False,
#     width=800,
#     height=500)
# fig.show()

In [ ]:
final_series = dfpivot[[ 'Illinois', 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'Wisconsin',
 'Wyoming']]

final_series.to_csv("Illinois_enf.csv")

# Texas

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO

In [ ]:
dfpivot.Texas.head(10)

time_value
2020-01-27    0.000000
2020-02-03    0.000000
2020-02-10    0.000000
2020-02-17    0.000000
2020-02-24    0.000000
2020-03-02    0.000000
2020-03-09    0.044277
2020-03-16    0.245225
2020-03-23    2.312611
2020-03-30    8.204829
Name: Texas, dtype: float64

In [ ]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-03-15")]
#dfpivot_cor = dfpivot[(dfpivot.index > "2020-5-4")&(dfpivot.index < "2021-6-30")]#


In [ ]:
pd.to_datetime("2021-03-22")+relativedelta(weeks=+8)

Timestamp('2021-05-17 00:00:00')

In [ ]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
               Alabama      Alaska  ...  Wisconsin    Wyoming
time_value                          ...                      
2021-03-15  165.334697  134.748143  ...  65.973386  72.811199

[1 rows x 52 columns]


In [ ]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Texas)
mi = mi[mi.Texas>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Texas
Alabama,0.854565
Alaska,0.926727
Arizona,0.974450
Arkansas,0.930758
California,0.857567
Colorado,0.831549
Connecticut,0.896226
Delaware,0.934231
District of Columbia,0.868491
Florida,0.978210


In [ ]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [ ]:
set(list(maskmandate.State))

{'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming'}

In [ ]:
final_series = dfpivot[[ 'Texas','Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'Tennessee',
 'Utah',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming']]
 
final_series.to_csv("Texas.csv")

# Louisiana reissue

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO

In [ ]:
dfpivot.Texas.head(10)

time_value
2020-01-27    0.000000
2020-02-03    0.000000
2020-02-10    0.000000
2020-02-17    0.000000
2020-02-24    0.000000
2020-03-02    0.000000
2020-03-09    0.044277
2020-03-16    0.245225
2020-03-23    2.312611
2020-03-30    8.204829
Name: Texas, dtype: float64

In [ ]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-08-09")]
#dfpivot_cor = dfpivot[(dfpivot.index > "2020-5-4")&(dfpivot.index < "2021-6-30")]#


In [ ]:
pd.to_datetime("2021-8-16")+relativedelta(weeks=+8)

Timestamp('2021-10-11 00:00:00')

In [ ]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
               Alabama      Alaska  ...   Wisconsin     Wyoming
time_value                          ...                        
2021-08-09  466.846502  302.284078  ...  145.662653  267.718537

[1 rows x 52 columns]


In [ ]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Louisiana)
mi = mi[mi.Louisiana>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Louisiana
Alabama,0.712461
Alaska,0.865775
Arizona,0.875565
Arkansas,0.859357
California,0.811898
Colorado,0.733303
Connecticut,0.847697
Delaware,0.863492
District of Columbia,0.847958
Florida,0.911590


In [ ]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [ ]:
set(list(maskmandate.State))

{'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Wisconsin',
 'Wyoming'}

In [ ]:
final_series = dfpivot[[ 'Louisiana','Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Wisconsin',
 'Wyoming']]
 
final_series.to_csv("Louisiana_enf.csv")

# Oregon reissue

In [6]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta,MO

In [7]:
dfpivot.Oregon.head(10)

time_value
2020-01-27    0.000000
2020-02-03    0.000000
2020-02-10    0.000000
2020-02-17    0.000000
2020-02-24    0.000000
2020-03-02    0.070730
2020-03-09    0.259342
2020-03-16    0.589413
2020-03-23    3.583632
2020-03-30    9.784258
Name: Oregon, dtype: float64

In [8]:
dfpivot_cor = dfpivot[(dfpivot.index >= "2020-03-16")&(dfpivot.index <= "2021-08-23")]


In [9]:
pd.to_datetime("2021-8-30")+relativedelta(weeks=+8)

Timestamp('2021-10-25 00:00:00')

In [10]:
print(dfpivot_cor.head(1))
print(dfpivot_cor.tail(1))

             Alabama    Alaska   Arizona  ...  West Virginia  Wisconsin   Wyoming
time_value                                ...                                    
2020-03-16  0.731479  0.138345  0.188644  ...            0.0   0.822953  0.343449

[1 rows x 52 columns]
               Alabama      Alaska  ...   Wisconsin    Wyoming
time_value                          ...                       
2021-08-23  574.821011  463.041102  ...  181.461101  449.57481

[1 rows x 52 columns]


In [11]:
maskmandate = pd.read_csv("/Statewidemaskorders.csv")
maskmandate[['Start_date', 'End_date']] = maskmandate['Effective'].str.split(' - ', expand=True)
maskmandate['Start_date']=pd.to_datetime(maskmandate['Start_date'])
maskmandate.End_date[maskmandate.End_date == 'present'] = "November 21, 2021"
maskmandate['End_date']=pd.to_datetime(maskmandate['End_date'])
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
             "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
             "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
             "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
             "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", 
             "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
             "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
nomandate = list(set(state_names) - set(maskmandate.State.values))
nomandatedf = pd.DataFrame(data={'State':nomandate})
maskmandate = maskmandate.append(nomandatedf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
nomandate

['Tennessee',
 'Missouri',
 'South Carolina',
 'Idaho',
 'Arizona',
 'Florida',
 'Georgia',
 'Nebraska',
 'Alaska',
 'Oklahoma',
 'South Dakota']

In [13]:
corr = dfpivot_cor.pct_change().dropna().corr()
mi = pd.DataFrame(corr.Oregon)
mi = mi[mi.Oregon>0.7]
mi.style.background_gradient(cmap='coolwarm')

,Oregon
Alaska,0.870293
Arizona,0.919890
Arkansas,0.859285
California,0.768228
Colorado,0.803373
Connecticut,0.855383
Delaware,0.866766
District of Columbia,0.773871
Florida,0.918844
Georgia,0.820094


In [14]:
maskmandate=maskmandate[(maskmandate.State.isin(mi.index))]#&(maskmandate.End_date>enddate)]

In [15]:
set(list(maskmandate.State))

{'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Wisconsin',
 'Wyoming'}

In [16]:
final_series = dfpivot[['Oregon','Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Pennsylvania',
 'South Carolina',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Wisconsin',
 'Wyoming']]
 
final_series.to_csv("Oregon_enf.csv")